In [1]:
#import required packages
import os
import ast
import urllib2
import urllib
import pandas as pd
import json
from cStringIO import StringIO
import scipy.stats
from scipy.cluster.vq import whiten
from matplotlib import gridspec
from sklearn.cluster import KMeans
import statsmodels.formula.api as smf
from pandas.tools.plotting import scatter_matrix
import geopandas as gpd
import glob
from fiona.crs import from_epsg
import datetime as dt
import zipfile
from matplotlib import cm
from descartes import PolygonPatch
import scipy.cluster as scp
import pylab as pl
from pandas.io.common import ZipFile
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from geopandas import GeoDataFrame
import seaborn as sns
from sklearn import cluster
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


## Task1: Calculate travel times of the complete route between the following Stop IDs:
- 553332 ROOSEVELT AV/61 ST NB
- 551576 74 ST/ROOSEVELT AV STATION NB
- 804110 LaGUARDIA INTERNAL RDWY/TERMINAL B NB
- 503875 LGA/TERMINAL D SB
- 503848 LGA/TERMINAL C SB
- 505192 ROOSEVELT AV/74 ST SB
- 982075 61 ST/ROOSEVELT AV SB

In [2]:
#read in data
mta = pd.read_csv('all-records_2017-12-10.csv')
mta.head()

,id,lat,lon,bearing,route,direction,service_date,gtfs_trip_id,gtfs_shape_id,gtfs_short_route_name,...,sched_arrive,next_predicted_arrival_time,next_predicted_departure_time,next_stop_distance_from_origin,next_stop_distance_to_vehicle,num_stops_from_call,stops_from_call_presentable,recorded_at_time,stop_distance_difference,at_stop_boolean
0,d254a85a-58dc-4d31-bdaf-ba343654cad9,40.801521,-73.968296,338.48288,MTA NYCT_M60+,1,2017-12-10,MTA NYCT_MQ_D7-Sunday-154300_SBS60_39,MTA_SBS600034,M60-SBS,...,26:22:00,2017-12-11 02:23:30.080000,2017-12-11 02:23:30.080000,15038.55,9.12,0,at stop,2017-12-11 02:23:07,2.609209,1
1,c5321b0c-a245-4a3a-aa94-4a177a917222,40.801521,-73.968296,338.48288,MTA NYCT_M60+,1,2017-12-10,MTA NYCT_MQ_D7-Sunday-154300_SBS60_39,MTA_SBS600034,M60-SBS,...,26:22:00,2017-12-11 02:23:00.100000,2017-12-11 02:23:00.100000,15038.55,9.12,0,at stop,2017-12-11 02:22:36,2.609209,1
2,7c125d31-3517-47d0-8e5c-b0ce1b2f052c,40.801521,-73.968296,338.48288,MTA NYCT_M60+,1,2017-12-10,MTA NYCT_MQ_D7-Sunday-154300_SBS60_39,MTA_SBS600034,M60-SBS,...,26:22:00,2017-12-11 02:22:30.027000,2017-12-11 02:22:30.027000,15038.55,9.12,0,at stop,2017-12-11 02:22:03,2.609209,1
3,6fba1c52-f3ec-499e-a82d-9e5362571030,40.801521,-73.968296,338.48288,MTA NYCT_M60+,1,2017-12-10,MTA NYCT_MQ_D7-Sunday-154300_SBS60_39,MTA_SBS600034,M60-SBS,...,26:22:00,2017-12-11 02:22:00.216000,2017-12-11 02:22:00.216000,15038.55,9.12,0,at stop,2017-12-11 02:21:31,2.609209,1
4,89586119-c515-428c-b851-46f8a4cff420,40.801750,-73.968527,234.76240,MTA NYCT_M60+,1,2017-12-10,MTA NYCT_MQ_D7-Sunday-154300_SBS60_39,MTA_SBS600034,M60-SBS,...,26:22:00,2017-12-11 02:21:00.004000,2017-12-11 02:21:00.004000,15038.55,50.16,0,approaching,2017-12-11 02:20:28,NaN,0


In [3]:
#check route
mta.route.unique()

array(['MTA NYCT_M60+', 'MTABC_Q70+'], dtype=object)

In [4]:
#selecting only entries in route Q70+
df = mta.loc[mta['route'] == 'MTABC_Q70+']

In [5]:
#check direction 
df.direction.unique()

array([0])

In [6]:
#check stops
df.origin_stop_id.unique()

array(['MTA_553332'], dtype=object)

In [7]:
df.destination_stop_id.unique()

array(['MTA_982075'], dtype=object)

In [8]:
df.next_stop_id.unique()

array(['MTA_982075', 'MTA_505192', 'MTA_804110', 'MTA_503875',
       'MTA_503848', 'MTA_551576', 'MTA_553332'], dtype=object)

In [9]:
df['recorded_at_time'] = pd.to_datetime(df['recorded_at_time'])
df['next_predicted_arrival_time'] = pd.to_datetime(df['next_predicted_arrival_time'])

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [10]:
df.sort_values(['recorded_at_time'],ascending=True, inplace=True)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
df.vehicle_id.unique()

array(['MTABC_7451', 'MTABC_7449', 'MTABC_7448', 'MTABC_7450',
       'MTABC_7453', 'MTABC_7446', 'MTABC_7445', 'MTABC_7472',
       'MTABC_7455', 'MTABC_7463'], dtype=object)

There are 10 buses running the same route

In [12]:
#select only required columns
df1 = df[['id', 'lat', 'lon', 'vehicle_id', 'next_stop_id','next_predicted_arrival_time', 'next_predicted_departure_time','next_stop_distance_from_origin', 'recorded_at_time', 'at_stop_boolean']]
df1.head(5)

,id,lat,lon,vehicle_id,next_stop_id,next_predicted_arrival_time,next_predicted_departure_time,next_stop_distance_from_origin,recorded_at_time,at_stop_boolean
34612,f958a26b-b017-4654-ae01-c23851c65a33,40.745688,-73.902816,MTABC_7451,MTA_553332,NaT,NaN,0.01,2017-12-09 23:58:07,1
34611,c9720755-0d32-48ed-9eb2-cfcb68d79ec3,40.745688,-73.902816,MTABC_7451,MTA_553332,NaT,NaN,0.01,2017-12-09 23:58:40,1
34610,f851c396-afdb-4ef8-b37b-93f7bbde6af2,40.745688,-73.902816,MTABC_7451,MTA_553332,NaT,NaN,0.01,2017-12-09 23:59:11,1
34609,5a2bdbe4-c4eb-4552-8230-bdfac12ba553,40.745688,-73.902816,MTABC_7451,MTA_553332,NaT,NaN,0.01,2017-12-09 23:59:42,1
34608,eca20a41-4761-4b58-b6cc-ea7453f01631,40.745688,-73.902816,MTABC_7451,MTA_553332,NaT,NaN,0.01,2017-12-09 23:59:42,1


In [13]:
df1['time'] = df1['recorded_at_time'].dt.time

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [14]:
df1['time'] = df1['time'].astype(str)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [15]:
df1['time'] = df1['time'].str.split(':').str[0]
df1['time'] = df1['time'].astype(int)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Sort times into intervals:

- 0-5
- 6-11
- 12-17
- 18-23

In [16]:
# sorting into time interval bins
df1['TIMEINT'] = pd.cut(df1['time'], bins=[0,5,12,18,300], labels=[1,2,3,4])
df1.head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,id,lat,lon,vehicle_id,next_stop_id,next_predicted_arrival_time,next_predicted_departure_time,next_stop_distance_from_origin,recorded_at_time,at_stop_boolean,time,TIMEINT
34612,f958a26b-b017-4654-ae01-c23851c65a33,40.745688,-73.902816,MTABC_7451,MTA_553332,NaT,NaN,0.01,2017-12-09 23:58:07,1,23,4
34611,c9720755-0d32-48ed-9eb2-cfcb68d79ec3,40.745688,-73.902816,MTABC_7451,MTA_553332,NaT,NaN,0.01,2017-12-09 23:58:40,1,23,4
34610,f851c396-afdb-4ef8-b37b-93f7bbde6af2,40.745688,-73.902816,MTABC_7451,MTA_553332,NaT,NaN,0.01,2017-12-09 23:59:11,1,23,4
34609,5a2bdbe4-c4eb-4552-8230-bdfac12ba553,40.745688,-73.902816,MTABC_7451,MTA_553332,NaT,NaN,0.01,2017-12-09 23:59:42,1,23,4
34608,eca20a41-4761-4b58-b6cc-ea7453f01631,40.745688,-73.902816,MTABC_7451,MTA_553332,NaT,NaN,0.01,2017-12-09 23:59:42,1,23,4


In [17]:
#df1[df1['TIMEINT'].isnull()]

In [18]:
df1['TIMEINT'].fillna(1, inplace=True)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/pandas/core/generic.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [19]:
df1 = df1.loc[df1['at_stop_boolean'] == 1]

# Calculate Duration by Time Interval Bins

- Filter to only include origin and destination stops. Calculate time delta between two time stamps

In [20]:
stops = ['MTA_553332', 'MTA_982075']
se = df1.loc[df1['next_stop_id'].isin(stops)]

In [21]:
dse = dict(tuple(se.groupby('vehicle_id')))

In [22]:
dse51 = dse['MTABC_7451']
dse51.reset_index(drop=True, inplace=True)
dse51['time_delta']=np.nan

dse49 = dse['MTABC_7449']
dse49.reset_index(drop=True, inplace=True)
dse49['time_delta']=np.nan

dse48 = dse['MTABC_7448']
dse48.reset_index(drop=True, inplace=True)
dse48['time_delta']=np.nan

dse50 = dse['MTABC_7450']
dse50.reset_index(drop=True, inplace=True)
dse50['time_delta']=np.nan

dse53 = dse['MTABC_7453']
dse53.reset_index(drop=True, inplace=True)
dse53['time_delta']=np.nan

dse46 = dse['MTABC_7446']
dse46.reset_index(drop=True, inplace=True)
dse46['time_delta']=np.nan

dse45 = dse['MTABC_7445']
dse45.reset_index(drop=True, inplace=True)
dse45['time_delta']=np.nan

dse72 = dse['MTABC_7472']
dse72.reset_index(drop=True, inplace=True)
dse72['time_delta']=np.nan

dse55 = dse['MTABC_7455']
dse55.reset_index(drop=True, inplace=True)
dse55['time_delta']=np.nan

dse63 = dse['MTABC_7463']
dse63.reset_index(drop=True, inplace=True)
dse63['time_delta']=np.nan


In [23]:
for i in range(0,(len(dse51) - 1)):
    dse51['time_delta'][i] = dse51.recorded_at_time[i+1] - dse51.recorded_at_time[i]
dse51;

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [24]:
for i in range(0,(len(dse49) - 1)):
    dse49['time_delta'][i] = dse49.recorded_at_time[i+1] - dse49.recorded_at_time[i]
dse49;

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [25]:
for i in range(0,(len(dse48) - 1)):
    dse48['time_delta'][i] = dse48.recorded_at_time[i+1] - dse48.recorded_at_time[i]
dse48;

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [26]:
for i in range(0,(len(dse50) - 1)):
    dse50['time_delta'][i] = dse50.recorded_at_time[i+1] - dse50.recorded_at_time[i]
dse50;

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [27]:
for i in range(0,(len(dse53) - 1)):
    dse53['time_delta'][i] = dse53.recorded_at_time[i+1] - dse53.recorded_at_time[i]
dse53;

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [28]:
for i in range(0,(len(dse46) - 1)):
    dse46['time_delta'][i] = dse46.recorded_at_time[i+1] - dse46.recorded_at_time[i]
dse46;

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [29]:
for i in range(0,(len(dse45) - 1)):
    dse45['time_delta'][i] = dse45.recorded_at_time[i+1] - dse45.recorded_at_time[i]
dse45;

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [30]:
for i in range(0,(len(dse72) - 1)):
    dse72['time_delta'][i] = dse72.recorded_at_time[i+1] - dse72.recorded_at_time[i]
dse72;

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [31]:
for i in range(0,(len(dse55) - 1)):
    dse55['time_delta'][i] = dse55.recorded_at_time[i+1] - dse55.recorded_at_time[i]
dse55;

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [32]:
for i in range(0,(len(dse63) - 1)):
    dse63['time_delta'][i] = dse63.recorded_at_time[i+1] - dse63.recorded_at_time[i]
dse63;

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [33]:
dse51['time_delta'] = pd.to_timedelta(dse51['time_delta']).astype('timedelta64[s]')
dse51 = dse51.loc[(dse51['time_delta'] > 500) & (dse51['time_delta'] < 3600)]
#d51['delay'] = (d51['recorded_at_time'] - d51['next_predicted_arrival_time']).astype('timedelta64[s]')

dse49['time_delta'] = pd.to_timedelta(dse49['time_delta']).astype('timedelta64[s]')
dse49 = dse49.loc[(dse49['time_delta'] > 500) & (dse49['time_delta'] < 3600)]
#d49['delay'] = (d49['recorded_at_time'] - d49['next_predicted_arrival_time']).astype('timedelta64[s]')

dse48['time_delta'] = pd.to_timedelta(dse48['time_delta']).astype('timedelta64[s]')
dse48 = dse48.loc[(dse48['time_delta'] > 500) & (dse48['time_delta'] < 3600)]
#d48['delay'] = (d48['recorded_at_time'] - d48['next_predicted_arrival_time']).astype('timedelta64[s]')

dse50['time_delta'] = pd.to_timedelta(dse50['time_delta']).astype('timedelta64[s]')
dse50 = dse50.loc[(dse50['time_delta'] > 500) & (dse50['time_delta'] < 3600)]
#d50['delay'] = (d50['recorded_at_time'] - d50['next_predicted_arrival_time']).astype('timedelta64[s]')

dse53['time_delta'] = pd.to_timedelta(dse53['time_delta']).astype('timedelta64[s]')
dse53 = dse53.loc[(dse53['time_delta'] > 500) & (dse53['time_delta'] < 3600)]
#d53['delay'] = (d53['recorded_at_time'] - d53['next_predicted_arrival_time']).astype('timedelta64[s]')

dse46['time_delta'] = pd.to_timedelta(dse46['time_delta']).astype('timedelta64[s]')
dse46 = dse46.loc[(dse46['time_delta'] > 500) & (dse46['time_delta'] < 3600)]
#d46['delay'] = (d46['recorded_at_time'] - d46['next_predicted_arrival_time']).astype('timedelta64[s]')

dse45['time_delta'] = pd.to_timedelta(dse45['time_delta']).astype('timedelta64[s]')
dse45 = dse45.loc[(dse45['time_delta'] > 500) & (dse45['time_delta'] < 3600)]
#d45['delay'] = (d45['recorded_at_time'] - d45['next_predicted_arrival_time']).astype('timedelta64[s]')

dse72['time_delta'] = pd.to_timedelta(dse72['time_delta']).astype('timedelta64[s]')
dse72 = dse72.loc[(dse72['time_delta'] > 500) & (dse72['time_delta'] < 3600)]
#d72['delay'] = (d72['recorded_at_time'] - d72['next_predicted_arrival_time']).astype('timedelta64[s]')

dse55['time_delta'] = pd.to_timedelta(dse55['time_delta']).astype('timedelta64[s]')
dse55 = dse55.loc[(dse55['time_delta'] > 500) & (dse55['time_delta'] < 3600)]
#d55['delay'] = (d55['recorded_at_time'] - d55['next_predicted_arrival_time']).astype('timedelta64[s]')

dse63['time_delta'] = pd.to_timedelta(dse63['time_delta']).astype('timedelta64[s]')
dse63 = dse63.loc[(dse63['time_delta'] > 500) & (dse63['time_delta'] < 3600)]
#d63['delay'] = (d63['recorded_at_time'] - d63['next_predicted_arrival_time']).astype('timedelta64[s]')

In [34]:
dsebus = pd.concat([dse51, dse49, dse48, dse50, dse53, dse46, dse45, dse72, dse55, dse63], ignore_index=True)

# Calculate duration of route by time intervals

In [35]:
avg_time = dsebus.groupby(['TIMEINT']).mean()

In [36]:
avg_time = avg_time[['time_delta']].reset_index()

In [37]:
avg_time['Duration_in_mins'] = avg_time['time_delta'] / 60

In [38]:
avg_time

,TIMEINT,time_delta,Duration_in_mins
0,1,2172.923077,36.215385
1,2,2664.772727,44.412879
2,3,3053.125000,50.885417
3,4,2599.250000,43.320833
